[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_commodities_comparison/EMQA_commodities_comparison.ipynb)

# EMQA_commodities_comparison

Comparing oil and gas prices.

**Output:** `lecture1_commodities_comparison.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
brent = fetch('BZ=F'); gas = fetch('NG=F')
df = pd.DataFrame({'Brent': brent, 'Gas': gas}).dropna()
norm = df / df.iloc[0] * 100

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9))
ax1.plot(norm.index, norm['Brent'], color=COLORS['blue'], lw=1, label='Brent Oil')
ax1.plot(norm.index, norm['Gas'], color=COLORS['green'], lw=1, label='Natural Gas')
ax1.axhline(100, color='gray', ls='--', lw=0.5)
ax1.set_title('Normalized Prices (Base=100)', fontweight='bold')
ax1.set_ylabel('Index')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), ncol=2, frameon=False)

bret = np.log(df['Brent']/df['Brent'].shift(1)).dropna()
gret = np.log(df['Gas']/df['Gas'].shift(1)).dropna()
rcorr = bret.rolling(60).corr(gret)
ax2.plot(rcorr.index, rcorr, color=COLORS['purple'], lw=1)
ax2.axhline(0, color='gray', ls='--', lw=0.5)
ax2.set_title('60-Day Rolling Correlation (Oil vs Gas Returns)', fontweight='bold')
ax2.set_ylabel('Correlation')
ax2.legend(['Rolling Correlation'], loc='upper center', bbox_to_anchor=(0.5, -0.08), frameon=False)

fig.tight_layout()
save_fig(fig, 'lecture1_commodities_comparison.pdf')
plt.show()
